In [22]:
# AE 498 HW 1 MOID Calculation

import numpy as np
from scipy.optimize import root

AU_KM = 149597870.7  # Conversion factor


# Rotation matrices

def rot_z(t):
    c, s = np.cos(t), np.sin(t)
    return np.array([[c,-s,0],
                     [s, c,0],
                     [0, 0,1]])

def rot_x(t):
    c, s = np.cos(t), np.sin(t)
    return np.array([[1,0,0],
                     [0,c,-s],
                     [0,s, c]])



def orbit_frame(a, e, i, w, O):
    i, w, O = np.radians([i, w, O])
    R = rot_z(O) @ rot_x(i) @ rot_z(w)
    p = a * (1 - e**2)
    return R, p, e


# Position and derivative relating to true anomaly

def position_and_derivative(f, R, p, e):

    r = p / (1 + e*np.cos(f))

    x_pf = r*np.cos(f)
    y_pf = r*np.sin(f)

    dr_df = (p*e*np.sin(f)) / (1 + e*np.cos(f))**2
    dx_df = dr_df*np.cos(f) - r*np.sin(f)
    dy_df = dr_df*np.sin(f) + r*np.cos(f)


    r_vec  = R @ np.array([x_pf, y_pf, 0])
    dr_vec = R @ np.array([dx_df, dy_df, 0])

    return r_vec, dr_vec


# Gronchi solver implementation

def moid_gronchi_style(el1, el2):

    R1, p1, e1 = orbit_frame(*el1)
    R2, p2, e2 = orbit_frame(*el2)

    def system(vars):
        f1, f2 = vars
        r1, dr1 = position_and_derivative(f1, R1, p1, e1)
        r2, dr2 = position_and_derivative(f2, R2, p2, e2)

        d = r1 - r2
        eq1 = np.dot(d, dr1)
        eq2 = -np.dot(d, dr2)

        return [eq1, eq2]

    # grid
    
    guesses = []
    grid = np.linspace(0, 2*np.pi, 8)
    for g1 in grid:
        for g2 in grid:
            guesses.append((g1, g2))

    solutions = []

    for g in guesses:
        sol = root(system, g, tol=1e-12)

        if sol.success:
            f1, f2 = sol.x

            # Normalize angles
            f1 = np.mod(f1, 2*np.pi)
            f2 = np.mod(f2, 2*np.pi)

            r1,_ = position_and_derivative(f1, R1, p1, e1)
            r2,_ = position_and_derivative(f2, R2, p2, e2)
            d = np.linalg.norm(r1 - r2)

            solutions.append((d, f1, f2))

    if not solutions:
        return None

    # Remove duplicate solutions
    
    unique = []
    for sol in solutions:
        if not any(abs(sol[0] - u[0]) < 1e-3 for u in unique):
            unique.append(sol)

    best = min(unique, key=lambda x: x[0])
    return best


# Orbital parameters given in HW 1 table


earth = (
    1.4765067E+08,
    9.1669995E-03,
    4.2422693E-03,
    6.64375167E+01,
    1.4760836E+01
)

apophis = (
    1.3793939E+08,
    1.9097084E-01,
    3.3356539E+00,
    1.2919949E+02,
    2.0381969E+02
)

yr4 = (
    3.7680703E+08,
    6.6164147E-01,
    3.4001497E+00,
    1.3429905E+02,
    2.7147904E+02
)

atlas = (
    -3.9552667E+07,
    6.1469268E+00,
    1.7512507E+02,
    1.2817255E+02,
    3.2228906E+02
)

class style:
    BOLD = '\033[1m'
    END = '\033[0m'


cases = [
    (style.BOLD + "Earth – 99942 Apophis:" + style.END, earth, apophis),
    (style.BOLD + "Earth – 2024 YR4:" + style.END, earth, yr4),
    (style.BOLD + "Apophis – 2024 YR4:" + style.END, apophis, yr4),
    (style.BOLD + "Earth – 3I/ATLAS:" + style.END, earth, atlas)
]

for name, o1, o2 in cases:

    result = moid_gronchi_style(o1, o2)

    if result is None:
        print(f"{name}: No solution found")
        continue

    d_km, f1, f2 = result

    print("\n", name)
    print() 
    print("MOID (km):", d_km)
    print() 
    print("MOID (AU):", d_km/AU_KM)
    print() 
    print("f1 (deg):", np.degrees(f1))
    print() 
    print("f2 (deg):", np.degrees(f2)) 
    print('-' * 28)
    print()



 Earth – 99942 Apophis:

MOID (km): 847896.668544713

MOID (AU): 0.005667839151568305

f1 (deg): 127.96743162532977

f2 (deg): 236.15494627361602
----------------------------


 Earth – 2024 YR4:

MOID (km): 264452.64426733454

MOID (AU): 0.0017677567403192628

f1 (deg): 11.92699380634437

f2 (deg): 47.350193935258716
----------------------------


 Apophis – 2024 YR4:

MOID (km): 7530657.644111849

MOID (AU): 0.0503393371100425

f1 (deg): 124.29705073413356

f2 (deg): 51.506008072196956
----------------------------


 Earth – 3I/ATLAS:

MOID (km): 56610142.751206756

MOID (AU): 0.3784154312244951

f1 (deg): 112.72958218992015

f2 (deg): 0.2187246712778486
----------------------------

